In [1]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import os
import csv
import textract
import nltk
import joblib

import MeCab
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from konlpy.tag import *
from collections import Counter
from nltk.corpus import stopwords
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline

In [2]:
df = pd.read_csv('adoor_data/answers.csv', encoding='UTF8')
df['created_at'] = pd.to_datetime(df['created_at'])

start_date = pd.Timestamp(2019, 1, 28, 0)
end_date = pd.Timestamp(2019, 4, 1, 0)

mask = (df['created_at'] > start_date) & (df['created_at'] <= end_date)
df = df.loc[mask]

df.head()

,id,author_id,question_id,content,tag_string,created_at,updated_at
0,1,5,236,"오늘 상담쌤과 칭찬에 대한 대화를 나눴다. 내가 애들의 칭찬을 잘 못 믿는, 그리고...",NaN,2019-01-28 14:11:42.543635,2019-01-28 14:11:42.543635
1,2,5,103,원하는 순간에 생각의 흐름을 멈추고 하고자 하는 일에 집중할 수 있는 능력.,NaN,2019-01-28 14:12:09.623446,2019-01-28 14:12:09.623446
2,3,5,98,가만히 공상/유튜브. 혼자 있는 시간.,NaN,2019-01-28 14:13:16.547800,2019-01-28 14:13:16.547800
3,4,5,95,게으름. 그 중에서도 요즘 가장 싫은건 게으름으로 인해 더러워진 방. 너무 짜증나서...,NaN,2019-01-28 14:14:38.838070,2019-01-28 14:14:38.838070
4,6,4,5,진수니랑 같이 프론트 더 멋있게 만들기!!!!!\r\n,NaN,2019-01-28 15:55:14.088330,2019-01-28 15:55:14.088330


In [3]:
def getNVM_lemma(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    pos = []
    tags = ['NNG','NNP','VV','VA', 'VX', 'VCP','VCN']
    for word_ in word_tag[:-2]:
        word = word_.split("\t")
        tag = word[1].split(",")
        if(len(word[0]) < 2) or ("게" in word[0]):
            continue
        if(tag[-1] != '*'):
            t = tag[-1].split('/')
            if(len(t[0]) > 1 and ('VV' in t[1] or 'VA' in t[1] or 'VX' in t[1])):
                pos.append(t[0])
        else:
            if(tag[0] in tags):
                pos.append(word[0])
    return pos

In [8]:
tf_vect = CountVectorizer(tokenizer=getNVM_lemma, min_df=2, max_df=6000, max_features=25000)
dtm = tf_vect.fit_transform(df['content'].values.astype('U'))

n_topics = 5

lda = LatentDirichletAllocation(n_components=n_topics, topic_word_prior=0.01, doc_topic_prior=0.001)
lda.fit(dtm)
saved_model = joblib.dump(dtm, 'LDA_IP.pkl')

In [9]:
names = tf_vect.get_feature_names()
topics_word = dict()
n_words = 10

for idx, topic in enumerate(lda.components_):
    vocab = []
    for i in topic.argsort()[:-(n_words-1):-1]:
        vocab.append((names[i], topic[i].round(2)))
    topics_word[idx+1] = [(names[i], topic[i].round(2)) for i in topic.argsort()[:-(n_words-1):-1]]
max_dict = dict()
for idx, vec in enumerate(lda.transform(dtm)):
    t = vec.argmax()
    if(t not in max_dict):
        max_dict[t] = (vec[t], idx)
    else:
        if(max_dict[t][0] < vec[t]):
            max_dict[t] = (vec[t], idx)
            
sorted_review = sorted(max_dict.items(), key = lambda x: x[0], reverse=False)

for key, value in sorted_review:
    print('주제 {}: {}'.format(key+1, topics_word[key+1]))
    print('[주제 {}의 대표 글 :{}]\n{}\n\n'.format(key+1, value[0], df['content'].values.astype('U')[value[1]]))

주제 1: [('행복', 37.64), ('생각', 29.25), ('사람', 21.1), ('친구', 21.05), ('질문', 19.83), ('모르', 18.99), ('시간', 18.08), ('대하', 17.57)]
[주제 1의 대표 리뷰 :0.9999436659390181]
이 질문을 보고 내가 진짜 싫어하는 소확행, 힐링을 강요하는 사회 뭐 이런 게 생각나서 궤변을 한 번 지어내 봤당 물론 질문 자체에는 전혀 문제가 없지만!! 내가 이 질문에서 불편함을 느끼게 하는 (그래서 멀쩡한 질문에 답은 안 하고 이상한 소리만 하게 만드는) 숨겨진 맥락은 분명히 존재한다. 사실은 내가 평소에 평화라는 거에 대해 생각을 안 하고 사는 아무 생각이
없는 사람이라 대답할 말이 생각 안 나서 이런 걸 쓰는 걸 수도 있당...ㅎ.ㅎ.ㅎ.ㅎ.ㅎ 사실 얼마 전에 비슷한 논지의 글을 봐서 그거의 내용은 다 가져다 쓰면서 논리는 1도 없는 글인 걸 알아서 부끄럽지만 여기는 어도어니까~   

이 사악한 질문은 두 가지를 숨기고 있다.
1. 평화라는 말은 애초에 한 개인에게 적용할 수 없는 단어임에도 불구하고, 주어를 1인칭으로 설정하고 구어체를 사용함으로써 평화라는 말을 개인이 처한 상황과 관련지어 생각하도록 유도한다. 
2. 따라서 이 질문을 받은 개인은 여기서 평화라는 말을 평온 또는 행복과 비슷한 의미를 가진 것으로 왜곡해서 생각하게 되고, 세 단어가 실제로는 모두 다른 의미의 단어임에도 불구하고 그것들의 의미를 서로 혼동해 '조용하고 만족스러운' 정도의 평화도 평온도 행복도 아닌 어중간하고 부족한(?) 것을 답하고 그것을 평화고 평온이며 행복이라고 받아들인다. 하지만 그 답변은 셋 중 무엇도 아니다. 
이런 암시적이고 편향적인 질문에 넘어가서 이 질문이 의도하는 종류의 대답을 해 버리고 싶지는 않다. 힐링은 아프기 때문에 할 수 있는 것이고, 대부분의 소확행은 강요되는 것이다. 


주제 2: [('사람', 53.91), ('생각', 38.29), ('그러', 19.

In [10]:
visual = pyLDAvis.sklearn.prepare(lda_model=lda, dtm=dtm, vectorizer=tf_vect)
pyLDAvis.save_html(visual, 'LDA_Visualization2.html')
pyLDAvis.display(visual)